Dataset

Here is a description of the data set, which has three columns with the names of the images. There is a probability and whether the label is single crystal or twin crystal. There are only four types of probabilities, 0,0.3, 0.6, and 1. So this can be turned into a multi-classification task. Next came the task of dividing the data set. First, I used all the data sets for training, but the effect was obviously not very good. Later, I also divided the data set, taking the latter 200 hundred data as the verification set, and all the previous data as the training and test questions. After the division of all data sets was completed, I divided the data set according to label: mano and poly, and verified the last 100 pieces of each data







explore ways to deal with class imbalance:(Here's what the training section requires)

Because there are four different probabilities in the data. So you can divide all the probabilities into four labels. But because each probability is given a different number, there is an imbalance in the data. For the data imbalance, I can do a pre-processing on him. The usual pre-processing used is resampling, and I used resampling here. Found good effect

Since it is a classification task, SVM is a good model for classification tasks. But because the SVM model is very simple, my innovation is to do a feature engineering of the picture. First, I SIFT the images. However, the effect was not good, although I added the selection of key points after adjusting the contrast and brightness of the images in sift method. But the results are still not very good

Then I carried out hog processing on the image, and adjusted the contrast and brightness, but the brightness of contrast did not have a particularly obvious effect on hog feature extraction. However, hog has the best effect compared with all image feature engineering extraction.

Then I also processed the image with OTSU. The performance effect is not as good as hog

As for why you use these methods, you can use chat to understand. These are just a few common image manipulation methods, so I didn't specifically study why they were chosen

Finally, since both random forest and logistic regression are used for classification tasks, I have also written two moduli for comparison with SVM, and the results are inferior to SVM

Discussion part: 

I think the otsu method is a failure, because the accuracy obtained is lower than the original one. If you want to know the reason, you can ask chat, but I haven't explored it further

SVM

The most basic SVM model starts with four classification tasks, and the probability is divided into four categories of labels. Image feature processing can flatten the image directly, but the result of SVM adjustment is poor

In [1]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3



images, probs, types= load_dataset()
y = np.array([map_probability_to_label(prob) for prob in probs])
X = images.reshape(images.shape[0], -1)
# Split dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[910  64  28 129]
 [112  67   7  35]
 [ 52   7   2  19]
 [191  53  26 266]]
              precision    recall  f1-score   support

           0       0.72      0.80      0.76      1131
           1       0.35      0.30      0.33       221
           2       0.03      0.03      0.03        80
           3       0.59      0.50      0.54       536

    accuracy                           0.63      1968
   macro avg       0.42      0.41      0.41      1968
weighted avg       0.62      0.63      0.62      1968



Preprocessing of data sets：Oversampling(explore ways to deal with class imbalance.)

Oversampling is a common method when dealing with data sets with unbalanced categories. The Oversampling results are much better, but it takes a lot of time

In [2]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import numpy as np
from elpv_reader import load_dataset

def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3

images, probs, types = load_dataset()
y = np.array([map_probability_to_label(prob) for prob in probs])
X = images.reshape(images.shape[0], -1)

# Over-sampling using RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.75, random_state=42)

# Create and train the SVM classifier
svm_classifier = svm.SVC(C=0.5, kernel='linear', gamma=0.008, decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)

# Predictions and evaluation
y_pred = svm_classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[ 749  165   77  150]
 [  77  980   14   33]
 [  10   13 1121    0]
 [ 220  111   93  711]]
              precision    recall  f1-score   support

           0       0.71      0.66      0.68      1141
           1       0.77      0.89      0.83      1104
           2       0.86      0.98      0.92      1144
           3       0.80      0.63      0.70      1135

    accuracy                           0.79      4524
   macro avg       0.78      0.79      0.78      4524
weighted avg       0.78      0.79      0.78      4524



Test after hog feature extraction after oversampling

In [7]:
from sklearn import svm
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog
import cv2
# Read image
def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd

def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3

images, probs, types = load_dataset()
y = np.array([map_probability_to_label(prob) for prob in probs])
X = np.array([hog_feature_extraction(image) for image in images])
# X = images.reshape(images.shape[0], -1)

# Over-sampling using RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.75, random_state=42)

# Create and train the SVM classifier
svm_classifier = svm.SVC(C=0.5, kernel='linear', gamma=0.008, decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)

# Predictions and evaluation
y_pred = svm_classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[ 843  122   36  140]
 [ 152  919    0   33]
 [  10   23 1111    0]
 [ 184   46   37  868]]
              precision    recall  f1-score   support

           0       0.71      0.74      0.72      1141
           1       0.83      0.83      0.83      1104
           2       0.94      0.97      0.95      1144
           3       0.83      0.76      0.80      1135

    accuracy                           0.83      4524
   macro avg       0.83      0.83      0.83      4524
weighted avg       0.83      0.83      0.83      4524



Image feature extraction

HOG

Feature extraction is carried out by hog

The best effect

In [3]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog
import cv2
# Read image
def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd
  
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by hog
X = np.array([hog_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1003   21    8   99]
 [ 172   27    0   22]
 [  55    4    1   20]
 [ 208   24    2  302]]
              precision    recall  f1-score   support

           0       0.70      0.89      0.78      1131
           1       0.36      0.12      0.18       221
           2       0.09      0.01      0.02        80
           3       0.68      0.56      0.62       536

    accuracy                           0.68      1968
   macro avg       0.46      0.40      0.40      1968
weighted avg       0.63      0.68      0.64      1968



Comparison of random forest and SVM

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog
import cv2

def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd
  
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by hog
X = np.array([hog_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
# Use RandomForest classifiers
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


y_pred_rf = rf_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

[[1001    1    0  129]
 [ 190    0    0   31]
 [  56    0    0   24]
 [ 226    1    0  309]]
              precision    recall  f1-score   support

           0       0.68      0.89      0.77      1131
           1       0.00      0.00      0.00       221
           2       0.00      0.00      0.00        80
           3       0.63      0.58      0.60       536

    accuracy                           0.67      1968
   macro avg       0.33      0.37      0.34      1968
weighted avg       0.56      0.67      0.61      1968



c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Comparison of logistic regression and SVM

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog
import cv2

def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd
  
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by hog
X = np.array([hog_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
# Use logistic regression classifiers
lr_classifier = LogisticRegression(random_state=42, max_iter=1000)
lr_classifier.fit(X_train, y_train)

y_pred_lr = lr_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

[[1023    9    6   93]
 [ 176   22    1   22]
 [  54    2    1   23]
 [ 222   16    3  295]]
              precision    recall  f1-score   support

           0       0.69      0.90      0.79      1131
           1       0.45      0.10      0.16       221
           2       0.09      0.01      0.02        80
           3       0.68      0.55      0.61       536

    accuracy                           0.68      1968
   macro avg       0.48      0.39      0.39      1968
weighted avg       0.64      0.68      0.64      1968



SIFT

sift is used to extract image features without increasing contrast and brightness

In [4]:

import cv2
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from elpv_reader import load_dataset
import numpy as np

def sift_feature_extraction(image, num_keypoints=100):
    sift_class = cv2.SIFT_create()
    keypts_l, keypts_d = sift_class.detectAndCompute(image, None)
    
    # # If fewer than num_keypoints are detected, fill them with zeros
    if keypts_d is not None:
        if keypts_d.shape[0] < num_keypoints:
            keypts_d = np.pad(keypts_d, ((0, num_keypoints - keypts_d.shape[0]), (0, 0)), 'constant')
        
        else:
            keypts_d = keypts_d[:num_keypoints, :] # Truncate if more than num_keypoints are detected
    else:
        # If no feature points are detected, an all-zero matrix is created
        keypts_d = np.zeros((num_keypoints, 128))  

    return keypts_d

def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()


y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by sift
proc = np.array([sift_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X = proc.reshape(proc.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[856  61   4 210]
 [160  19   2  40]
 [ 57   3   1  19]
 [273  23   2 238]]
              precision    recall  f1-score   support

           0       0.64      0.76      0.69      1131
           1       0.18      0.09      0.12       221
           2       0.11      0.01      0.02        80
           3       0.47      0.44      0.46       536

    accuracy                           0.57      1968
   macro avg       0.35      0.32      0.32      1968
weighted avg       0.52      0.57      0.54      1968




Still used SIFT feature extraction,but increase brightness and contrast, and the number of selected key points to compare. There is no significant change

In [4]:

import cv2
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from elpv_reader import load_dataset
import numpy as np



def sift_feature_extraction(image, num_keypoints=500):
    adjusted = cv2.convertScaleAbs(image, alpha=2, beta=25)
    sift_class = cv2.SIFT_create()
    keypts_l, keypts_d = sift_class.detectAndCompute(adjusted, None)
    
    # If fewer than num_keypoints are detected, fill them with zeros
    if keypts_d is not None:
        if keypts_d.shape[0] < num_keypoints:
            keypts_d = np.pad(keypts_d, ((0, num_keypoints - keypts_d.shape[0]), (0, 0)), 'constant')
        else:
            # Truncate if more than num_keypoints are detected
            keypts_d = keypts_d[:num_keypoints, :]
    else:
        # If no feature points are detected, an all-zero matrix is created
        keypts_d = np.zeros((num_keypoints, 128))

    return keypts_d
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by sift
proc = np.array([sift_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X = proc.reshape(proc.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[835 102  13 181]
 [162  30   2  27]
 [ 48  12   1  19]
 [286  48  11 191]]
              precision    recall  f1-score   support

           0       0.63      0.74      0.68      1131
           1       0.16      0.14      0.15       221
           2       0.04      0.01      0.02        80
           3       0.46      0.36      0.40       536

    accuracy                           0.54      1968
   macro avg       0.32      0.31      0.31      1968
weighted avg       0.50      0.54      0.52      1968



OTSU

Using otsu to extract image features, the effect is normal

In [6]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from elpv_reader import load_dataset

def otsu_feature_extraction(image):
#  otsu fuction
    def extract_features(image):
        # Apply Otsu thresholding
        otsu_threshold = otsu(image)

        # Binaryize the image
        _, binary_image = cv2.threshold(image, otsu_threshold, 255, cv2.THRESH_BINARY)

        return binary_image

    def otsu(image):
        hist, _ = np.histogram(image.ravel(), 256, [0,256])
        total = image.size
        current_max, threshold = 0, 0
        total_sum = np.sum(np.arange(256) * hist)
        bg_sum, fg_sum, bg_weight, fg_weight = 0, 0, 0, 0
        for i in range(256):
            bg_weight += hist[i]
            fg_weight = total - bg_weight
            if bg_weight == 0 or fg_weight == 0:
                continue
            bg_sum += i * hist[i]
            fg_sum = total_sum - bg_sum
            bg_mean = bg_sum / bg_weight
            fg_mean = fg_sum / fg_weight
            # Calculate between class variance
            var_between = bg_weight * fg_weight * (bg_mean - fg_mean) ** 2
            # Check if new maximum found
            if var_between > current_max:
                current_max = var_between
                threshold = i
        return threshold
    
    features = extract_features(image)
    return features    


def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
#Process all images by otsu
proc = np.array([otsu_feature_extraction(image) for image in images])

# Split dataset into training and testing sets
X = proc.reshape(proc.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[930  30  14 157]
 [141  48   6  26]
 [ 57   5   3  15]
 [215  27   6 288]]
              precision    recall  f1-score   support

           0       0.69      0.82      0.75      1131
           1       0.44      0.22      0.29       221
           2       0.10      0.04      0.06        80
           3       0.59      0.54      0.56       536

    accuracy                           0.64      1968
   macro avg       0.46      0.40      0.42      1968
weighted avg       0.61      0.64      0.62      1968



Take 200 data sets as verification sets

In [7]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


# Assuming you have a function to load your dataset
images, probs, types= load_dataset()

y = np.array([map_probability_to_label(prob) for prob in probs])
X = images.reshape(images.shape[0], -1)
# Split dataset into training and testing sets
X_val = X[-200:]
y_val = y[-200:]
X_train_test = X[:-200]
y_train_test = y[:-200]
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.75, random_state=42,stratify=y_train_test)
svm_classifier = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier.fit(X_train, y_train)


y_pred = svm_classifier.predict(X_test)

# Calculate confusion matrix and classification report
print("data classification Report:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
y_pred_val = svm_classifier.predict(X_val)
print("Verify the performance of the set:")
print(confusion_matrix(y_val, y_pred_val))
print(classification_report(y_val, y_pred_val))

data classification Report:
[[787  64  31 141]
 [104  65   5  24]
 [ 40  11   5  21]
 [199  58   7 256]]
              precision    recall  f1-score   support

           0       0.70      0.77      0.73      1023
           1       0.33      0.33      0.33       198
           2       0.10      0.06      0.08        77
           3       0.58      0.49      0.53       520

    accuracy                           0.61      1818
   macro avg       0.43      0.41      0.42      1818
weighted avg       0.60      0.61      0.60      1818

Verify the performance of the set:
[[139   1   0   4]
 [ 29   2   0   0]
 [  4   0   0   0]
 [ 18   0   3   0]]
              precision    recall  f1-score   support

           0       0.73      0.97      0.83       144
           1       0.67      0.06      0.12        31
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00        21

    accuracy                           0.70       200
   macro avg       0.3

The data set was divided according to the label and divided into mono and poly respectively for training, and the effect was better

In [8]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset

   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


images, probs, types = load_dataset()  

# Convert probability to category label
y = np.array([map_probability_to_label(prob) for prob in probs])

# Split the data set by type
mono_indices = types == 'mono'
poly_indices = types == 'poly'

# mono
images_mono = images[mono_indices]
y_mono = y[mono_indices]
images_mono_f = images_mono.reshape(images_mono.shape[0], -1)
images_mono_val = images_mono_f[-100:]
y_mono_val = y_mono[-100:]
images_mono_train_test = images_mono_f[:-100]
y_mono_train_test = y_mono[:-100]

# poly
images_poly = images[poly_indices]
y_poly = y[poly_indices]
images_poly_f = images_poly.reshape(images_poly.shape[0], -1)
images_poly_val = images_poly_f[-100:]
y_poly_val = y_poly[-100:]
images_poly_train_test = images_poly_f[:-100]
y_poly_train_test = y_poly[:-100]

# Process mono crystal data separately
X_train_mono, X_test_mono, y_train_mono, y_test_mono = train_test_split(images_mono_train_test, y_mono_train_test, test_size=0.75, random_state=42,stratify=y_mono_train_test)

# Train the SVM classifier
svm_classifier_mono = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_mono.fit(X_train_mono, y_train_mono)

# Process poly crystal data separately
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(images_poly_train_test, y_poly_train_test, test_size=0.75, random_state=42,stratify=y_poly_train_test)

# Train the SVM classifier
svm_classifier_poly = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_poly.fit(X_train_poly, y_train_poly)

# Predict and evaluate single crystal data
y_pred_mono = svm_classifier_mono.predict(X_test_mono)
print("mono data classification Report:")
print(confusion_matrix(y_test_mono, y_pred_mono))
print(classification_report(y_test_mono, y_pred_mono))

y_mono_valp = svm_classifier_mono.predict(images_mono_val)
print("Verify the mono performance of the set:")
print(confusion_matrix(y_mono_val, y_mono_valp))
print(classification_report(y_mono_val, y_mono_valp))

# Prediction and evaluation of polycrystalline data
y_pred_poly = svm_classifier_poly.predict(X_test_poly)
print("poly data classification Report:")
print(confusion_matrix(y_test_poly, y_pred_poly))
print(classification_report(y_test_poly, y_pred_poly))

y_poly_valp = svm_classifier_poly.predict(images_poly_val)
print("Verify the poly performance of the set:")
print(confusion_matrix(y_poly_val, y_poly_valp))
print(classification_report(y_poly_val, y_poly_valp))

mono data classification Report:
[[344  20   7  16]
 [ 42  30   3   2]
 [ 24   0   6   7]
 [100  19  17  94]]
              precision    recall  f1-score   support

           0       0.67      0.89      0.77       387
           1       0.43      0.39      0.41        77
           2       0.18      0.16      0.17        37
           3       0.79      0.41      0.54       230

    accuracy                           0.65       731
   macro avg       0.52      0.46      0.47       731
weighted avg       0.66      0.65      0.63       731

Verify the mono performance of the set:
[[70  0  0  2]
 [14  0  0  1]
 [ 6  0  0  0]
 [ 7  0  0  0]]
              precision    recall  f1-score   support

           0       0.72      0.97      0.83        72
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         7

    accuracy                           0.70       100
   macro avg       0.18     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

poly data classification Report:
[[515  55   7  70]
 [ 51  37   2  22]
 [ 23   2   2   9]
 [119  24  14 136]]
              precision    recall  f1-score   support

           0       0.73      0.80      0.76       647
           1       0.31      0.33      0.32       112
           2       0.08      0.06      0.07        36
           3       0.57      0.46      0.51       293

    accuracy                           0.63      1088
   macro avg       0.42      0.41      0.42      1088
weighted avg       0.62      0.63      0.63      1088

Verify the poly performance of the set:
[[56  2  0  0]
 [17 12  0  0]
 [ 2  0  0  0]
 [ 9  1  0  1]]
              precision    recall  f1-score   support

           0       0.67      0.97      0.79        58
           1       0.80      0.41      0.55        29
           2       0.00      0.00      0.00         2
           3       1.00      0.09      0.17        11

    accuracy                           0.69       100
   macro avg       0.62     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Oversample for 2 dataset

In [9]:

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from imblearn.over_sampling import RandomOverSampler
   
def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


images, probs, types = load_dataset()  

# Convert probability to category label
y = np.array([map_probability_to_label(prob) for prob in probs])

# Split the data set by type
mono_indices = types == 'mono'
poly_indices = types == 'poly'

# mono
images_mono = images[mono_indices]
y_mono = y[mono_indices]
images_mono_f = images_mono.reshape(images_mono.shape[0], -1)
images_mono_val = images_mono_f[-100:]
y_mono_val = y_mono[-100:]
images_mono_train_test = images_mono_f[:-100]
y_mono_train_test = y_mono[:-100]

ros = RandomOverSampler(random_state=42)
X_mono_resampled, y_mono_resampled = ros.fit_resample(images_mono_train_test, y_mono_train_test)
X_mono_train, X_mono_test, y_mono_train, y_mono_test = train_test_split(X_mono_resampled, y_mono_resampled, test_size=0.75, random_state=42)


# poly
images_poly = images[poly_indices]
y_poly = y[poly_indices]
images_poly_f = images_poly.reshape(images_poly.shape[0], -1)
images_poly_val = images_poly_f[-100:]
y_poly_val = y_poly[-100:]
images_poly_train_test = images_poly_f[:-100]
y_poly_train_test = y_poly[:-100]

ros2 = RandomOverSampler(random_state=42)
X_poly_resampled, y_poly_resampled = ros2.fit_resample(images_poly_train_test, y_poly_train_test)
X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(X_poly_resampled, y_poly_resampled, test_size=0.75, random_state=42)

# Process mono crystal data separately
 # Train the SVM classifier
svm_classifier_mono = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_mono.fit(X_mono_train, y_mono_train)

# Process poly crystal data separately
# Train the SVM classifier
svm_classifier_poly = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_poly.fit(X_poly_train, y_poly_train)

# Predict and evaluate single crystal data
y_pred_mono = svm_classifier_mono.predict(X_mono_test)
print("mono data classification Report:")
print(confusion_matrix(y_mono_test, y_pred_mono))
print(classification_report(y_mono_test, y_pred_mono))

y_mono_valp = svm_classifier_mono.predict(images_mono_val)
print("Verify the mono performance of the set:")
print(confusion_matrix(y_mono_val, y_mono_valp))
print(classification_report(y_mono_val, y_mono_valp))

# Prediction and evaluation of polycrystalline data
y_pred_poly = svm_classifier_poly.predict(X_poly_test)
print("poly data classification Report:")
print(confusion_matrix(y_poly_test, y_pred_poly))
print(classification_report(y_poly_test, y_pred_poly))

y_poly_valp = svm_classifier_poly.predict(images_poly_val)
print("Verify the poly performance of the set:")
print(confusion_matrix(y_poly_val, y_poly_valp))
print(classification_report(y_poly_val, y_poly_valp))


mono data classification Report:
[[280  39  35  38]
 [ 42 314  21  10]
 [ 21   0 356   5]
 [ 81  40  24 242]]
              precision    recall  f1-score   support

           0       0.66      0.71      0.69       392
           1       0.80      0.81      0.81       387
           2       0.82      0.93      0.87       382
           3       0.82      0.63      0.71       387

    accuracy                           0.77      1548
   macro avg       0.77      0.77      0.77      1548
weighted avg       0.77      0.77      0.77      1548

Verify the mono performance of the set:
[[63  3  5  1]
 [12  0  1  2]
 [ 6  0  0  0]
 [ 5  0  2  0]]
              precision    recall  f1-score   support

           0       0.73      0.88      0.80        72
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         7

    accuracy                           0.63       100
   macro avg       0.18     

HOG for seperate dataset

HOG is used to extract features from images, and the data sets with different labels are trained respectively

This shows the best results

In [1]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog


# Read image
def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd

def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3


images, probs, types = load_dataset()  

# Convert probability to category label
y = np.array([map_probability_to_label(prob) for prob in probs])
images = np.array([hog_feature_extraction(image) for image in images])
# Split the data set by type
mono_indices = types == 'mono'
poly_indices = types == 'poly'

# mono
images_mono = images[mono_indices]
y_mono = y[mono_indices]
images_mono_f = images_mono.reshape(images_mono.shape[0], -1)
images_mono_val = images_mono_f[-100:]
y_mono_val = y_mono[-100:]
images_mono_train_test = images_mono_f[:-100]
y_mono_train_test = y_mono[:-100]

# poly
images_poly = images[poly_indices]
y_poly = y[poly_indices]
images_poly_f = images_poly.reshape(images_poly.shape[0], -1)
images_poly_val = images_poly_f[-100:]
y_poly_val = y_poly[-100:]
images_poly_train_test = images_poly_f[:-100]
y_poly_train_test = y_poly[:-100]

# Process single crystal data separately
X_train_mono, X_test_mono, y_train_mono, y_test_mono = train_test_split(images_mono_train_test, y_mono_train_test, test_size=0.75, random_state=42,stratify=y_mono_train_test)

# # Train the SVM classifier
svm_classifier_mono = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_mono.fit(X_train_mono, y_train_mono)

# Process polycrystalline data separately
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(images_poly_train_test, y_poly_train_test, test_size=0.75, random_state=42,stratify=y_poly_train_test)

# Train the SVM classifier
svm_classifier_poly = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_poly.fit(X_train_poly, y_train_poly)

# Predict and evaluate single crystal data
y_pred_mono = svm_classifier_mono.predict(X_test_mono)
print("mono data classification Report:")
print(confusion_matrix(y_test_mono, y_pred_mono))
print(classification_report(y_test_mono, y_pred_mono))

y_mono_valp = svm_classifier_mono.predict(images_mono_val)
print("Verify the mono performance of the set:")
print(confusion_matrix(y_mono_val, y_mono_valp))
print(classification_report(y_mono_val, y_mono_valp))

# Prediction and evaluation of polycrystalline data
y_pred_poly = svm_classifier_poly.predict(X_test_poly)
print("poly data classification Report:")
print(confusion_matrix(y_test_poly, y_pred_poly))
print(classification_report(y_test_poly, y_pred_poly))

y_poly_valp = svm_classifier_poly.predict(images_poly_val)
print("Verify the poly performance of the set:")
print(confusion_matrix(y_poly_val, y_poly_valp))
print(classification_report(y_poly_val, y_poly_valp))

mono data classification Report:
[[368   7   0  12]
 [ 51  24   0   2]
 [ 32   0   1   4]
 [ 74   2   3 151]]
              precision    recall  f1-score   support

           0       0.70      0.95      0.81       387
           1       0.73      0.31      0.44        77
           2       0.25      0.03      0.05        37
           3       0.89      0.66      0.76       230

    accuracy                           0.74       731
   macro avg       0.64      0.49      0.51       731
weighted avg       0.74      0.74      0.71       731

Verify the mono performance of the set:
[[72  0  0  0]
 [15  0  0  0]
 [ 6  0  0  0]
 [ 7  0  0  0]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84        72
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         7

    accuracy                           0.72       100
   macro avg       0.18     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

poly data classification Report:
[[583  10   1  53]
 [ 78  19   0  15]
 [ 25   1   2   8]
 [137   7   1 148]]
              precision    recall  f1-score   support

           0       0.71      0.90      0.79       647
           1       0.51      0.17      0.26       112
           2       0.50      0.06      0.10        36
           3       0.66      0.51      0.57       293

    accuracy                           0.69      1088
   macro avg       0.60      0.41      0.43      1088
weighted avg       0.67      0.69      0.66      1088

Verify the poly performance of the set:
[[58  0  0  0]
 [29  0  0  0]
 [ 2  0  0  0]
 [10  0  0  1]]
              precision    recall  f1-score   support

           0       0.59      1.00      0.74        58
           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00         2
           3       1.00      0.09      0.17        11

    accuracy                           0.59       100
   macro avg       0.40     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Test after hog feature extraction after oversampling after seperating dataset

In [9]:
from sklearn import svm
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from elpv_reader import load_dataset
from skimage.feature import hog
import cv2

# Read image
def hog_feature_extraction(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it is already a grayscale image, no conversion is required
    

# Calculate the HOG feature of the image
    fd, hog_image = hog(gray_image, orientations=4, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return fd

def map_probability_to_label(prob):
    if prob == 0:
        return 0
    elif 0 < prob <= 0.4:
        return 1
    elif 0.4 < prob <= 0.8:
        return 2
    else:
        return 3

images, probs, types = load_dataset()
y = np.array([map_probability_to_label(prob) for prob in probs])
images = np.array([hog_feature_extraction(image) for image in images])
# Convert probability to category label

# Split the data set by type
mono_indices = types == 'mono'
poly_indices = types == 'poly'

# mono
images_mono = images[mono_indices]
y_mono = y[mono_indices]
images_mono_f = images_mono.reshape(images_mono.shape[0], -1)
images_mono_val = images_mono_f[-100:]
y_mono_val = y_mono[-100:]
images_mono_train_test = images_mono_f[:-100]
y_mono_train_test = y_mono[:-100]

ros = RandomOverSampler(random_state=42)
X_mono_resampled, y_mono_resampled = ros.fit_resample(images_mono_train_test, y_mono_train_test)
X_mono_train, X_mono_test, y_mono_train, y_mono_test = train_test_split(X_mono_resampled, y_mono_resampled, test_size=0.75, random_state=42)


# poly
images_poly = images[poly_indices]
y_poly = y[poly_indices]
images_poly_f = images_poly.reshape(images_poly.shape[0], -1)
images_poly_val = images_poly_f[-100:]
y_poly_val = y_poly[-100:]
images_poly_train_test = images_poly_f[:-100]
y_poly_train_test = y_poly[:-100]

ros2 = RandomOverSampler(random_state=42)
X_poly_resampled, y_poly_resampled = ros2.fit_resample(images_poly_train_test, y_poly_train_test)
X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(X_poly_resampled, y_poly_resampled, test_size=0.75, random_state=42)

# Process mono crystal data separately
 # Train the SVM classifier
svm_classifier_mono = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_mono.fit(X_mono_train, y_mono_train)

# Process poly crystal data separately
# Train the SVM classifier
svm_classifier_poly = svm.SVC(C=0.5 ,kernel='linear',gamma=0.008,decision_function_shape='ovo')
svm_classifier_poly.fit(X_poly_train, y_poly_train)

# Predict and evaluate single crystal data
y_pred_mono = svm_classifier_mono.predict(X_mono_test)
print("mono data classification Report:")
print(confusion_matrix(y_mono_test, y_pred_mono))
print(classification_report(y_mono_test, y_pred_mono))

y_mono_valp = svm_classifier_mono.predict(images_mono_val)
print("Verify the mono performance of the set:")
print(confusion_matrix(y_mono_val, y_mono_valp))
print(classification_report(y_mono_val, y_mono_valp))

# Prediction and evaluation of polycrystalline data
y_pred_poly = svm_classifier_poly.predict(X_poly_test)
print("poly data classification Report:")
print(confusion_matrix(y_poly_test, y_pred_poly))
print(classification_report(y_poly_test, y_pred_poly))

y_poly_valp = svm_classifier_poly.predict(images_poly_val)
print("Verify the poly performance of the set:")
print(confusion_matrix(y_poly_val, y_poly_valp))
print(classification_report(y_poly_val, y_poly_valp))


mono data classification Report:
[[313  38  16  25]
 [ 46 323  11   7]
 [ 21   0 356   5]
 [ 70   3  14 300]]
              precision    recall  f1-score   support

           0       0.70      0.80      0.74       392
           1       0.89      0.83      0.86       387
           2       0.90      0.93      0.91       382
           3       0.89      0.78      0.83       387

    accuracy                           0.83      1548
   macro avg       0.84      0.84      0.84      1548
weighted avg       0.84      0.83      0.84      1548

Verify the mono performance of the set:
[[72  0  0  0]
 [15  0  0  0]
 [ 6  0  0  0]
 [ 6  0  0  1]]
              precision    recall  f1-score   support

           0       0.73      1.00      0.84        72
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         6
           3       1.00      0.14      0.25         7

    accuracy                           0.73       100
   macro avg       0.43     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

poly data classification Report:
[[480  76   7  92]
 [ 41 574   0  19]
 [  0   0 649   0]
 [135  53   8 452]]
              precision    recall  f1-score   support

           0       0.73      0.73      0.73       655
           1       0.82      0.91      0.86       634
           2       0.98      1.00      0.99       649
           3       0.80      0.70      0.75       648

    accuracy                           0.83      2586
   macro avg       0.83      0.83      0.83      2586
weighted avg       0.83      0.83      0.83      2586

Verify the poly performance of the set:
[[44 14  0  0]
 [22  7  0  0]
 [ 2  0  0  0]
 [ 6  2  0  3]]
              precision    recall  f1-score   support

           0       0.59      0.76      0.67        58
           1       0.30      0.24      0.27        29
           2       0.00      0.00      0.00         2
           3       1.00      0.27      0.43        11

    accuracy                           0.54       100
   macro avg       0.47     

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera